In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/'Colab Notebooks'/X.pickle .
!cp drive/'My Drive'/'Colab Notebooks'/y.pickle .

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time



# we separate memory for few sessions
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# load data from pickle files
X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

# normalize our data
# 255 is maximum value for pixel data
X = X / 255.0

# we found the best options for our model
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, 
                                                   layer_size, 
                                                   dense_layer,
                                                   int(time.time()))
      tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))      

      # initialize our model
      model = Sequential()
      # add layers to our model
      model.add(Conv2D(layer_size, (3, 3), input_shape = X.shape[1:]))
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size=(2, 2)))
      
      for l in range(conv_layer - 1):
        
        model.add(Conv2D(layer_size, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
      model.add(Flatten())  
      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))

      model.add(Dense(1))
      model.add(Activation('sigmoid'))

      model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

      model.fit(X, y, 
                batch_size=32, 
                epochs=10, 
                validation_split=0.3, 
                callbacks=[tensorboard])

model.save('64x3-CNN.model')

W0814 12:25:42.279183 140607164110720 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 12:25:42.446387 140607164110720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 17462 samples, validate on 7484 samples
Epoch 1/10
   32/17462 [..............................] - ETA: 17:30 - loss: 0.7116 - acc: 0.4062

W0814 12:25:45.434952 140607164110720 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (0.171161). Check your callbacks.


17462/17462 [==============================] - 27s 2ms/sample - loss: 0.6384 - acc: 0.6248 - val_loss: 0.5657 - val_acc: 0.7092
Epoch 2/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.5141 - acc: 0.7484 - val_loss: 0.4923 - val_acc: 0.7692
Epoch 3/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.4476 - acc: 0.7900 - val_loss: 0.4430 - val_acc: 0.7964
Epoch 4/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.3982 - acc: 0.8191 - val_loss: 0.4109 - val_acc: 0.8157
Epoch 5/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.3603 - acc: 0.8409 - val_loss: 0.3930 - val_acc: 0.8227
Epoch 6/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.3282 - acc: 0.8547 - val_loss: 0.4087 - val_acc: 0.8148
Epoch 7/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.2942 - acc: 0.8726 - val_loss: 0.3649 - val_acc: 0.8401
Epoch 8/10
17462/17462 [==============

In [0]:
!cp drive/'My Drive'/'Colab Notebooks'/Data/cat.jpg .
!cp drive/'My Drive'/'Colab Notebooks'/Data/dog.jpg .

In [9]:
import cv2
import tensorflow as tf

CATEGORIES = ['Dog', 'Cat']

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

mode = tf.keras.models.load_model('64x3-CNN.model')

prediction = model.predict([prepare('dog.jpg')])

print(CATEGORIES[int(prediction[0][0])])

Dog


In [13]:
prediction = model.predict([prepare('dog.jpg')])

print(CATEGORIES[int(prediction[0][0])])

Dog


In [14]:
prediction = model.predict([prepare('cat.jpg')])

print(CATEGORIES[int(prediction[0][0])])

Cat
